# Make a Good Model for the mood of the song

In this document I need to work more with NLP of the track lyrics and use columns that I discarded in the MVP and I also need to do  some feature engineering.

Better model = countries<BR />
Best model = also looking at number of streams and position, also looking at time on top list<BR />

Then when we have the best model we can use our predictions and decide the mood of a country and the mood of an artist. Then we can say what artist is suitable for what country. 

## Import stuff

In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
import sklearn.metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Load data

In [2]:
data = pd.read_csv('./data_top10c_more_lyrics.csv')

data.head(5)

,Unnamed: 0,Position,Streams,Track Name,Artist,ID,Date,Year,Month,Day,Country,Region,Lyrics,Acousticness,Energy,Instrumentalness,Mode,Tempo,Valence
0,0,177,40381,Bye Bye Bye,*NSYNC,4r8lRYnoOGdEi6YyI5OC1o,2017-10-05,2017,10,5,gb,eu,"hey, hey bye bye bye, bye bye bye bye i'm doi...",0.0408,0.928,0.00104,0.0,172.656,0.879
1,1,151,24132,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-23,2017,12,23,it,eu,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756
2,2,78,49766,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-24,2017,12,24,it,eu,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756
3,3,65,40725,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-25,2017,12,25,it,eu,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756
4,4,197,215739,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-22,2017,12,22,us,na,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756


## Fix a little bit with the data

### Look at each country individually

In [7]:
data['Country'].unique()

array(['gb', 'it', 'us', 'de', 'ca', 'nl', 'au', 'fr'], dtype=object)

In [9]:
# filter by country and make 8 new data frames
gb_data = data[data['Country'] == 'gb']
it_data = data[data['Country'] == 'it']
us_data = data[data['Country'] == 'us']
de_data = data[data['Country'] == 'de']
ca_data = data[data['Country'] == 'ca']
nl_data = data[data['Country'] == 'nl']
au_data = data[data['Country'] == 'au']
fr_data = data[data['Country'] == 'fr']

**Drop rows that are duplicates and keep only one row for each song**

In [10]:
gb_data_per_song = gb_data.drop_duplicates(subset=['Track Name'], keep='first')
it_data_per_song = it_data.drop_duplicates(subset=['Track Name'], keep='first')
us_data_per_song = us_data.drop_duplicates(subset=['Track Name'], keep='first')
de_data_per_song = de_data.drop_duplicates(subset=['Track Name'], keep='first')
ca_data_per_song = ca_data.drop_duplicates(subset=['Track Name'], keep='first')
nl_data_per_song = nl_data.drop_duplicates(subset=['Track Name'], keep='first')
au_data_per_song = au_data.drop_duplicates(subset=['Track Name'], keep='first')
fr_data_per_song = fr_data.drop_duplicates(subset=['Track Name'], keep='first')

**Drop all columns that might change per song**

In [11]:
gb_nlp_data = gb_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
it_nlp_data = it_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
us_nlp_data = us_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
de_nlp_data = de_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
ca_nlp_data = ca_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
nl_nlp_data = nl_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
au_nlp_data = au_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
fr_nlp_data = fr_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)

**Drop rows that have missing values in the Lyrics column**<BR />
We can use dropna to drop all rows that has missing values (should mostly be the Lyrics column)

In [13]:
gb_clean = gb_nlp_data.dropna(axis=0, how='any')
it_clean = it_nlp_data.dropna(axis=0, how='any')
us_clean = us_nlp_data.dropna(axis=0, how='any')
de_clean = de_nlp_data.dropna(axis=0, how='any')
ca_clean = ca_nlp_data.dropna(axis=0, how='any')
nl_clean = nl_nlp_data.dropna(axis=0, how='any')
au_clean = au_nlp_data.dropna(axis=0, how='any')
fr_clean = fr_nlp_data.dropna(axis=0, how='any')

### TextBlob

**Turn the lyrics in the Lyrics column into string**

In [14]:
gb_clean['Lyrics'] = gb_clean['Lyrics'].astype(str)
it_clean['Lyrics'] = it_clean['Lyrics'].astype(str)
us_clean['Lyrics'] = us_clean['Lyrics'].astype(str)
de_clean['Lyrics'] = de_clean['Lyrics'].astype(str)
ca_clean['Lyrics'] = ca_clean['Lyrics'].astype(str)
nl_clean['Lyrics'] = nl_clean['Lyrics'].astype(str)
au_clean['Lyrics'] = au_clean['Lyrics'].astype(str)
fr_clean['Lyrics'] = fr_clean['Lyrics'].astype(str)

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

**Make and run function for TextBlob on the Lyrics**

In [15]:
def sentiment_func(lyrics):
    try:
        return TextBlob(lyrics).sentiment
    except:
        return None

In [16]:
gb_clean['pol_sub'] = gb_clean['Lyrics'].apply(sentiment_func)
it_clean['pol_sub'] = it_clean['Lyrics'].apply(sentiment_func)
us_clean['pol_sub'] = us_clean['Lyrics'].apply(sentiment_func)
de_clean['pol_sub'] = de_clean['Lyrics'].apply(sentiment_func)
ca_clean['pol_sub'] = ca_clean['Lyrics'].apply(sentiment_func)
nl_clean['pol_sub'] = nl_clean['Lyrics'].apply(sentiment_func)
au_clean['pol_sub'] = au_clean['Lyrics'].apply(sentiment_func)
fr_clean['pol_sub'] = fr_clean['Lyrics'].apply(sentiment_func)

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

**Split the pol_sub column into 2 new columns (Polarity, Subjectivity)**

In [24]:
gb_clean['Polarity'] = gb_clean['pol_sub'].apply(lambda x: x[0])
gb_clean['Subjectivity'] = gb_clean['pol_sub'].apply(lambda x: x[1])

it_clean['Polarity'] = it_clean['pol_sub'].apply(lambda x: x[0])
it_clean['Subjectivity'] = it_clean['pol_sub'].apply(lambda x: x[1])

us_clean['Polarity'] = us_clean['pol_sub'].apply(lambda x: x[0])
us_clean['Subjectivity'] = us_clean['pol_sub'].apply(lambda x: x[1])

de_clean['Polarity'] = de_clean['pol_sub'].apply(lambda x: x[0])
de_clean['Subjectivity'] = de_clean['pol_sub'].apply(lambda x: x[1])

ca_clean['Polarity'] = ca_clean['pol_sub'].apply(lambda x: x[0])
ca_clean['Subjectivity'] = ca_clean['pol_sub'].apply(lambda x: x[1])

nl_clean['Polarity'] = nl_clean['pol_sub'].apply(lambda x: x[0])
nl_clean['Subjectivity'] = nl_clean['pol_sub'].apply(lambda x: x[1])

au_clean['Polarity'] = au_clean['pol_sub'].apply(lambda x: x[0])
au_clean['Subjectivity'] = au_clean['pol_sub'].apply(lambda x: x[1])

fr_clean['Polarity'] = fr_clean['pol_sub'].apply(lambda x: x[0])
fr_clean['Subjectivity'] = fr_clean['pol_sub'].apply(lambda x: x[1])

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

**Drop the pol_sub column**

In [26]:
gb_clean = gb_clean.drop(['pol_sub'], axis=1)
it_clean = it_clean.drop(['pol_sub'], axis=1)
us_clean = us_clean.drop(['pol_sub'], axis=1)
de_clean = de_clean.drop(['pol_sub'], axis=1)
ca_clean = ca_clean.drop(['pol_sub'], axis=1)
nl_clean = nl_clean.drop(['pol_sub'], axis=1)
au_clean = au_clean.drop(['pol_sub'], axis=1)
fr_clean = fr_clean.drop(['pol_sub'], axis=1)

### Do a quick check of the entire data frame

In [27]:
gb_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041 entries, 0 to 578617
Data columns (total 12 columns):
Track Name          1041 non-null object
Artist              1041 non-null object
ID                  1041 non-null object
Lyrics              1041 non-null object
Acousticness        1041 non-null float64
Energy              1041 non-null float64
Instrumentalness    1041 non-null float64
Mode                1041 non-null float64
Tempo               1041 non-null float64
Valence             1041 non-null float64
Polarity            1041 non-null float64
Subjectivity        1041 non-null float64
dtypes: float64(8), object(4)
memory usage: 145.7+ KB


In [28]:
it_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 952 entries, 1 to 578920
Data columns (total 12 columns):
Track Name          952 non-null object
Artist              952 non-null object
ID                  952 non-null object
Lyrics              952 non-null object
Acousticness        952 non-null float64
Energy              952 non-null float64
Instrumentalness    952 non-null float64
Mode                952 non-null float64
Tempo               952 non-null float64
Valence             952 non-null float64
Polarity            952 non-null float64
Subjectivity        952 non-null float64
dtypes: float64(8), object(4)
memory usage: 136.7+ KB


In [29]:
us_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908 entries, 4 to 578926
Data columns (total 12 columns):
Track Name          908 non-null object
Artist              908 non-null object
ID                  908 non-null object
Lyrics              908 non-null object
Acousticness        908 non-null float64
Energy              908 non-null float64
Instrumentalness    908 non-null float64
Mode                908 non-null float64
Tempo               908 non-null float64
Valence             908 non-null float64
Polarity            908 non-null float64
Subjectivity        908 non-null float64
dtypes: float64(8), object(4)
memory usage: 92.2+ KB


In [30]:
de_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1020 entries, 8 to 577856
Data columns (total 12 columns):
Track Name          1020 non-null object
Artist              1020 non-null object
ID                  1020 non-null object
Lyrics              1020 non-null object
Acousticness        1020 non-null float64
Energy              1020 non-null float64
Instrumentalness    1020 non-null float64
Mode                1020 non-null float64
Tempo               1020 non-null float64
Valence             1020 non-null float64
Polarity            1020 non-null float64
Subjectivity        1020 non-null float64
dtypes: float64(8), object(4)
memory usage: 103.6+ KB


In [31]:
ca_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801 entries, 32 to 578615
Data columns (total 12 columns):
Track Name          801 non-null object
Artist              801 non-null object
ID                  801 non-null object
Lyrics              801 non-null object
Acousticness        801 non-null float64
Energy              801 non-null float64
Instrumentalness    801 non-null float64
Mode                801 non-null float64
Tempo               801 non-null float64
Valence             801 non-null float64
Polarity            801 non-null float64
Subjectivity        801 non-null float64
dtypes: float64(8), object(4)
memory usage: 81.4+ KB


In [32]:
nl_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 36 to 575613
Data columns (total 12 columns):
Track Name          887 non-null object
Artist              887 non-null object
ID                  887 non-null object
Lyrics              887 non-null object
Acousticness        887 non-null float64
Energy              887 non-null float64
Instrumentalness    887 non-null float64
Mode                887 non-null float64
Tempo               887 non-null float64
Valence             887 non-null float64
Polarity            887 non-null float64
Subjectivity        887 non-null float64
dtypes: float64(8), object(4)
memory usage: 90.1+ KB


In [33]:
au_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646 entries, 41 to 578619
Data columns (total 12 columns):
Track Name          646 non-null object
Artist              646 non-null object
ID                  646 non-null object
Lyrics              646 non-null object
Acousticness        646 non-null float64
Energy              646 non-null float64
Instrumentalness    646 non-null float64
Mode                646 non-null float64
Tempo               646 non-null float64
Valence             646 non-null float64
Polarity            646 non-null float64
Subjectivity        646 non-null float64
dtypes: float64(8), object(4)
memory usage: 65.6+ KB


In [34]:
fr_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1238 entries, 48 to 578929
Data columns (total 12 columns):
Track Name          1238 non-null object
Artist              1238 non-null object
ID                  1238 non-null object
Lyrics              1238 non-null object
Acousticness        1238 non-null float64
Energy              1238 non-null float64
Instrumentalness    1238 non-null float64
Mode                1238 non-null float64
Tempo               1238 non-null float64
Valence             1238 non-null float64
Polarity            1238 non-null float64
Subjectivity        1238 non-null float64
dtypes: float64(8), object(4)
memory usage: 125.7+ KB


## Train/Test-split

Divide the data into a train and a test set (with a test set of 25%, which is also default)

In [35]:
gb_dep   = gb_clean['Valence']
gb_indep = gb_clean
gb_indep_train, gb_indep_test, gb_dep_train, gb_dep_test = train_test_split(gb_indep, gb_dep, test_size = 0.25,
                                                                            random_state=24)

it_dep   = it_clean['Valence']
it_indep = it_clean
it_indep_train, it_indep_test, it_dep_train, it_dep_test = train_test_split(it_indep, it_dep, test_size = 0.25,
                                                                            random_state=24)

us_dep   = us_clean['Valence']
us_indep = us_clean
us_indep_train, us_indep_test, us_dep_train, us_dep_test = train_test_split(us_indep, us_dep, test_size = 0.25,
                                                                            random_state=24)

de_dep   = de_clean['Valence']
de_indep = de_clean
de_indep_train, de_indep_test, de_dep_train, de_dep_test = train_test_split(de_indep, de_dep, test_size = 0.25,
                                                                            random_state=24)

ca_dep   = ca_clean['Valence']
ca_indep = ca_clean
ca_indep_train, ca_indep_test, ca_dep_train, ca_dep_test = train_test_split(ca_indep, ca_dep, test_size = 0.25,
                                                                            random_state=24)

nl_dep   = nl_clean['Valence']
nl_indep = nl_clean
nl_indep_train, nl_indep_test, nl_dep_train, nl_dep_test = train_test_split(nl_indep, nl_dep, test_size = 0.25,
                                                                            random_state=24)

au_dep   = au_clean['Valence']
au_indep = au_clean
au_indep_train, au_indep_test, au_dep_train, au_dep_test = train_test_split(au_indep, au_dep, test_size = 0.25,
                                                                            random_state=24)

fr_dep   = fr_clean['Valence']
fr_indep = fr_clean
fr_indep_train, fr_indep_test, fr_dep_train, fr_dep_test = train_test_split(fr_indep, fr_dep, test_size = 0.25,
                                                                            random_state=24)

## NLP

In [44]:
# read in csv with stop words
stop_words_df = pd.read_csv('./stop_words.csv')

In [45]:
# turn the df to list
stop_words = stop_words_df['colummn'].tolist()

### CountVectorizer (use in model 1)

In [58]:
# instantiate the model
cvec = CountVectorizer(stop_words = stop_words, max_features = 1000) 

GB

In [59]:
# fit the count vectorizer with training data. 
cvec.fit(gb_indep_train['Lyrics'])
# transform X_train
gb_cvec = cvec.transform(gb_indep_train['Lyrics'])
# Turn the features into a data frame
gb_df  = pd.DataFrame(gb_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
gb_indep_train_cvec = pd.concat([gb_indep_train.reset_index(drop=True), gb_df], axis=1)
# transform X_test
gb_cvec2 = cvec.transform(gb_indep_test['Lyrics'])
# Turn the features into a data frame
gb_df2  = pd.DataFrame(gb_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
gb_indep_test_cvec = pd.concat([gb_indep_test.reset_index(drop=True), gb_df2 ], axis=1)

IT

In [60]:
# fit the count vectorizer with training data. 
cvec.fit(it_indep_train['Lyrics'])
# transform X_train
it_cvec = cvec.transform(it_indep_train['Lyrics'])
# Turn the features into a data frame
it_df  = pd.DataFrame(it_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
it_indep_train_cvec = pd.concat([it_indep_train.reset_index(drop=True), it_df], axis=1)
# transform X_test
it_cvec2 = cvec.transform(it_indep_test['Lyrics'])
# Turn the features into a data frame
it_df2  = pd.DataFrame(it_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
it_indep_test_cvec = pd.concat([it_indep_test.reset_index(drop=True), it_df2 ], axis=1)

US

In [61]:
# fit the count vectorizer with training data. 
cvec.fit(us_indep_train['Lyrics'])
# transform X_train
us_cvec = cvec.transform(us_indep_train['Lyrics'])
# Turn the features into a data frame
us_df  = pd.DataFrame(us_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
us_indep_train_cvec = pd.concat([us_indep_train.reset_index(drop=True), us_df], axis=1)
# transform X_test
us_cvec2 = cvec.transform(us_indep_test['Lyrics'])
# Turn the features into a data frame
us_df2  = pd.DataFrame(us_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
us_indep_test_cvec = pd.concat([us_indep_test.reset_index(drop=True), us_df2 ], axis=1)

DE

In [62]:
# fit the count vectorizer with training data. 
cvec.fit(de_indep_train['Lyrics'])
# transform X_train
de_cvec = cvec.transform(de_indep_train['Lyrics'])
# Turn the features into a data frame
de_df  = pd.DataFrame(de_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
de_indep_train_cvec = pd.concat([de_indep_train.reset_index(drop=True), de_df], axis=1)
# transform X_test
de_cvec2 = cvec.transform(de_indep_test['Lyrics'])
# Turn the features into a data frame
de_df2  = pd.DataFrame(de_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
de_indep_test_cvec = pd.concat([de_indep_test.reset_index(drop=True), de_df2 ], axis=1)

CA

In [63]:
# fit the count vectorizer with training data. 
cvec.fit(ca_indep_train['Lyrics'])
# transform X_train
ca_cvec = cvec.transform(ca_indep_train['Lyrics'])
# Turn the features into a data frame
ca_df  = pd.DataFrame(ca_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
ca_indep_train_cvec = pd.concat([ca_indep_train.reset_index(drop=True), ca_df], axis=1)
# transform X_test
ca_cvec2 = cvec.transform(ca_indep_test['Lyrics'])
# Turn the features into a data frame
ca_df2  = pd.DataFrame(ca_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
ca_indep_test_cvec = pd.concat([ca_indep_test.reset_index(drop=True), ca_df2 ], axis=1)

NL

In [64]:
# fit the count vectorizer with training data. 
cvec.fit(nl_indep_train['Lyrics'])
# transform X_train
nl_cvec = cvec.transform(nl_indep_train['Lyrics'])
# Turn the features into a data frame
nl_df  = pd.DataFrame(nl_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
nl_indep_train_cvec = pd.concat([nl_indep_train.reset_index(drop=True), nl_df], axis=1)
# transform X_test
nl_cvec2 = cvec.transform(nl_indep_test['Lyrics'])
# Turn the features into a data frame
nl_df2  = pd.DataFrame(nl_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
nl_indep_test_cvec = pd.concat([nl_indep_test.reset_index(drop=True), nl_df2 ], axis=1)

AU

In [65]:
# fit the count vectorizer with training data. 
cvec.fit(au_indep_train['Lyrics'])
# transform X_train
au_cvec = cvec.transform(au_indep_train['Lyrics'])
# Turn the features into a data frame
au_df  = pd.DataFrame(au_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
au_indep_train_cvec = pd.concat([au_indep_train.reset_index(drop=True), au_df], axis=1)
# transform X_test
au_cvec2 = cvec.transform(au_indep_test['Lyrics'])
# Turn the features into a data frame
au_df2  = pd.DataFrame(au_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
au_indep_test_cvec = pd.concat([au_indep_test.reset_index(drop=True), au_df2 ], axis=1)

FR

In [66]:
# fit the count vectorizer with training data. 
cvec.fit(fr_indep_train['Lyrics'])
# transform X_train
fr_cvec = cvec.transform(fr_indep_train['Lyrics'])
# Turn the features into a data frame
fr_df  = pd.DataFrame(fr_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
fr_indep_train_cvec = pd.concat([fr_indep_train.reset_index(drop=True), fr_df], axis=1)
# transform X_test
fr_cvec2 = cvec.transform(fr_indep_test['Lyrics'])
# Turn the features into a data frame
fr_df2  = pd.DataFrame(fr_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
fr_indep_test_cvec = pd.concat([fr_indep_test.reset_index(drop=True), fr_df2 ], axis=1)

### TF-IDF (use in model 2)

In [67]:
# instantiate the model
tvec = TfidfVectorizer(stop_words = stop_words, max_features = 1000) 

GB

In [68]:
# fit the count vectorizer with training data. 
tvec.fit(gb_indep_train['Lyrics'])
# transform X_train
gb_tvec = tvec.transform(gb_indep_train['Lyrics'])
# Turn the features into a data frame
gb_df  = pd.DataFrame(gb_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
gb_indep_train_tvec = pd.concat([gb_indep_train.reset_index(drop=True), gb_df], axis=1)
# transform X_test
gb_tvec2 = tvec.transform(gb_indep_test['Lyrics'])
# Turn the features into a data frame
gb_df2  = pd.DataFrame(gb_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
gb_indep_test_tvec = pd.concat([gb_indep_test.reset_index(drop=True), gb_df2 ], axis=1)

IT

In [69]:
# fit the count vectorizer with training data. 
tvec.fit(it_indep_train['Lyrics'])
# transform X_train
it_tvec = tvec.transform(it_indep_train['Lyrics'])
# Turn the features into a data frame
it_df  = pd.DataFrame(it_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
it_indep_train_tvec = pd.concat([it_indep_train.reset_index(drop=True), it_df], axis=1)
# transform X_test
it_tvec2 = tvec.transform(it_indep_test['Lyrics'])
# Turn the features into a data frame
it_df2  = pd.DataFrame(it_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
it_indep_test_tvec = pd.concat([it_indep_test.reset_index(drop=True), it_df2 ], axis=1)

US

In [70]:
# fit the count vectorizer with training data. 
tvec.fit(us_indep_train['Lyrics'])
# transform X_train
us_tvec = tvec.transform(us_indep_train['Lyrics'])
# Turn the features into a data frame
us_df  = pd.DataFrame(us_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
us_indep_train_tvec = pd.concat([us_indep_train.reset_index(drop=True), us_df], axis=1)
# transform X_test
us_tvec2 = tvec.transform(us_indep_test['Lyrics'])
# Turn the features into a data frame
us_df2  = pd.DataFrame(us_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
us_indep_test_tvec = pd.concat([us_indep_test.reset_index(drop=True), us_df2 ], axis=1)

DE

In [71]:
# fit the count vectorizer with training data. 
tvec.fit(de_indep_train['Lyrics'])
# transform X_train
de_tvec = tvec.transform(de_indep_train['Lyrics'])
# Turn the features into a data frame
de_df  = pd.DataFrame(de_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
de_indep_train_tvec = pd.concat([de_indep_train.reset_index(drop=True), de_df], axis=1)
# transform X_test
de_tvec2 = tvec.transform(de_indep_test['Lyrics'])
# Turn the features into a data frame
de_df2  = pd.DataFrame(de_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
de_indep_test_tvec = pd.concat([de_indep_test.reset_index(drop=True), de_df2 ], axis=1)

CA

In [72]:
# fit the count vectorizer with training data. 
tvec.fit(ca_indep_train['Lyrics'])
# transform X_train
ca_tvec = tvec.transform(ca_indep_train['Lyrics'])
# Turn the features into a data frame
ca_df  = pd.DataFrame(ca_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
ca_indep_train_tvec = pd.concat([ca_indep_train.reset_index(drop=True), ca_df], axis=1)
# transform X_test
ca_tvec2 = tvec.transform(ca_indep_test['Lyrics'])
# Turn the features into a data frame
ca_df2  = pd.DataFrame(ca_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
ca_indep_test_tvec = pd.concat([ca_indep_test.reset_index(drop=True), ca_df2 ], axis=1)

NL

In [73]:
# fit the count vectorizer with training data. 
tvec.fit(nl_indep_train['Lyrics'])
# transform X_train
nl_tvec = tvec.transform(nl_indep_train['Lyrics'])
# Turn the features into a data frame
nl_df  = pd.DataFrame(nl_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
nl_indep_train_tvec = pd.concat([nl_indep_train.reset_index(drop=True), nl_df], axis=1)
# transform X_test
nl_tvec2 = tvec.transform(nl_indep_test['Lyrics'])
# Turn the features into a data frame
nl_df2  = pd.DataFrame(nl_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
nl_indep_test_tvec = pd.concat([nl_indep_test.reset_index(drop=True), nl_df2 ], axis=1)

AU

In [74]:
# fit the count vectorizer with training data. 
tvec.fit(au_indep_train['Lyrics'])
# transform X_train
au_tvec = tvec.transform(au_indep_train['Lyrics'])
# Turn the features into a data frame
au_df  = pd.DataFrame(au_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
au_indep_train_tvec = pd.concat([au_indep_train.reset_index(drop=True), au_df], axis=1)
# transform X_test
au_tvec2 = tvec.transform(au_indep_test['Lyrics'])
# Turn the features into a data frame
au_df2  = pd.DataFrame(au_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
au_indep_test_tvec = pd.concat([au_indep_test.reset_index(drop=True), au_df2 ], axis=1)

FR

In [75]:
# fit the count vectorizer with training data. 
tvec.fit(fr_indep_train['Lyrics'])
# transform X_train
fr_tvec = tvec.transform(fr_indep_train['Lyrics'])
# Turn the features into a data frame
fr_df  = pd.DataFrame(fr_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
fr_indep_train_tvec = pd.concat([fr_indep_train.reset_index(drop=True), fr_df], axis=1)
# transform X_test
fr_tvec2 = tvec.transform(fr_indep_test['Lyrics'])
# Turn the features into a data frame
fr_df2  = pd.DataFrame(fr_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
fr_indep_test_tvec = pd.concat([fr_indep_test.reset_index(drop=True), fr_df2 ], axis=1)

## Models (Lasso) - CountVec

I did choose Lasso bc that gave the best score when looking at all countries togehter.<BR />
If the Lasso scores very well in one country (compared to the total score) then I might consider trying other models.

**Lasso function (best hyperparameters to use and evauation of the model)**

In [78]:
def get_best_hype(model, params, X_train, y_train):  
    # standardize
    ss = StandardScaler()
    ss.fit(X_train)
    X_train_s = ss.transform(X_train)
    
    # Best Hyperparameters
    gs = GridSearchCV(model, grid) # cv default = 3
    
    # fit
    gs.fit(X_train_s, y_train)
     
    return {'best_score': gs.best_score_,'best_params': gs.best_params_} 

def evaluate_model(model, X_train, X_test, y_train, y_test):
    # standardize the predictors
    ss = StandardScaler()
    ss.fit(X_train)
    X_train_s = ss.transform(X_train)
    X_test_s = ss.transform(X_test)
    
    # fit
    model.fit(X_train_s, y_train)
    
    # Evaluate: predict
    
    y_pred = model.predict(X_test_s)
    y_true = y_test
    
    mean_square_error = np.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    
    # Evaluate: score
    score = model.score(X_test_s, y_test)
    
    return {'Score (R^2)': score.mean(), 'MSE': mean_square_error}

**Great Brittan / United Kingdom**

In [84]:
# Declare indep and dep
X_train_gb = gb_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_gb = gb_dep_train
X_test_gb = gb_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_gb = gb_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_gb, y_train_gb)

{'best_params': {'alpha': 0.014174741629268055, 'selection': 'cyclic'},
 'best_score': 0.1683966517723151}

In [85]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0142, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_gb, X_test_gb, y_train_gb, y_test_gb)

{'MSE': 0.21918612906036872, 'Score (R^2)': 0.12372100674644926}

In [86]:
data['Country'].unique()

array(['gb', 'it', 'us', 'de', 'ca', 'nl', 'au', 'fr'], dtype=object)

**Italy**

**US**

**Deutchland / Germany**

**Canada**

**Netherlands**

**Australia**

**France**

## Models (Lasso) - TF-IDF

In [ ]:
gb_indep_train_tvec, gb_indep_test_tvec, gb_dep_train, gb_dep_test